In [1]:
import findspark

In [2]:
findspark.init('/home/daniel/spark-2.4.7-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [5]:
df = spark.read.json('/home/daniel/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/people.json')

In [6]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
# long - larger int data type
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
# list all columns in dataframe
df.columns

['age', 'name']

In [9]:
# NOTE: will not work with JAVA ver higher than 8 
# describe - get back a dataframe details
# append .show() to see the data with stat summary
df.describe()

DataFrame[summary: string, age: string, name: string]

In [10]:
from pyspark.sql.types import (StructField,StringType,
                               IntegerType,StructType)

In [11]:
# create a schema: pass in string (name), type (int), boolean true
data_schema = [StructField('age',IntegerType(),True),
               StructField('name',StringType(),True)]

In [12]:
# pass in data_schema to StructType
final_struc = StructType(fields=data_schema)

In [13]:
# read the json file but now with a new schema in the parameter
df = spark.read.json('/home/daniel/spark-2.4.7-bin-hadoop2.7/examples/src/main/resources/people.json', schema=final_struc)

In [14]:
# new schema has set long to integer for age
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



# DataFrame Basics Part 2

In [15]:
# check the type of the column (column object)
type(df['age'])

pyspark.sql.column.Column

In [16]:
# use the select method to see the results of the column
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [17]:
# if we check type of df.select we see that it returns df
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [18]:
# grab the top two rows
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [19]:
# now index the list [0]
df.head(2)[0]

Row(age=None, name='Michael')

In [20]:
# check the type - returns row object
type(df.head(2)[0])

pyspark.sql.types.Row

In [21]:
# select mulitple columns
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [22]:
# adding a new column (withColumn method)
df.withColumn('new_age',df['age']-2).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     28|
|  19| Justin|     17|
+----+-------+-------+



In [23]:
# original df is untouched
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [24]:
# rename column for the return df 
df.withColumnRenamed('age','my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [25]:
# Using SQL! 
# we need to first create a tempview called people
df.createOrReplaceTempView('people')

In [26]:
# spark sql using our tempview 'people'
results = spark.sql("SELECT * FROM people")

In [27]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [28]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")

In [29]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

